In [1]:
# ===================== 🔧 Встановлення бібліотек =====================
!pip install -U transformers accelerate sentence-transformers faiss-cpu gradio pymupdf

In [1]:
from huggingface_hub import login

login(token="")

In [ ]:
!nvidia-smi



/bin/bash: line 1: nvidia-smi: command not found


In [6]:
!pip install -U langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [5]:
!pip install langchain faiss-cpu openai gradio sentence-transformers PyMuPDF


In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"


In [12]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader

def load_and_process_document(file_path):
    loader = PyMuPDFLoader(file_path)
    pages = loader.load()

    # Додаємо номер сторінки до метаданих
    for i, page in enumerate(pages):
        page.metadata["page"] = i + 1

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(pages)

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embeddings)

    return vectorstore


In [14]:
def ask_specific_page(page_num):
    if not vectorstore:
        return "❗ Завантажте документ"

    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 4,
            "filter": {"page": page_num}  # <-- головне
        }
    )

    llm = ChatOpenAI(
        model="openai/gpt-4-1",
        temperature=0.3,
        max_tokens=512
    )

    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa.invoke({"query": f"What is written on page {page_num}?"})["result"]

def build_qa_chain(vectorstore):
    llm = ChatOpenAI(
        model="openai/gpt-4.1",  # OpenRouter GPT-4.1 ID
        temperature=0.3,
        max_tokens=512
    )

    retriever = vectorstore.as_retriever(search_type="similarity", k=3)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    return qa_chain


In [30]:
#  RAG-реалізація через OpenRouter (GPT-4.1), LangChain та Gradio

import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains.summarize import load_summarize_chain
import fitz  # PyMuPDF
import pdfplumber

# 📌 Налаштування OpenRouter
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# 🌍 Глобальні змінні
vectorstore = None
qa_chain = None
page_texts = {}

# 🔄 Завантаження та індексація документа

def load_and_process_document(file_path):
    global page_texts
    page_texts = {}
    pages = []

    # pdfplumber
    try:
        with pdfplumber.open(file_path) as pdf:
            for i, page in enumerate(pdf.pages, start=1):
                text = page.extract_text() or ""
                page_texts[i] = text
                content = f"Page {i}:\n{text}" if text else ""
                pages.append(Document(content, metadata={"page": i}))
    except Exception:
        # Фолбек PyMuPDF
        doc = fitz.open(file_path)
        for i, p in enumerate(doc, start=1):
            text = p.get_text() or ""
            page_texts[i] = text
            content = f"Page {i}:\n{text}" if text else ""
            pages.append(Document(content, metadata={"page": i}))

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(pages)
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.from_documents(chunks, embeddings)

# 🧠 Створення RAG-ланцюга

def build_qa_chain(vstore):
    retriever = vstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.3, max_tokens=512)
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 📥 Завантаження файлу

def upload_file(file):
    global vectorstore, qa_chain
    try:
        vectorstore = load_and_process_document(file.name)
        qa_chain = build_qa_chain(vectorstore)
        return "✅ Документ проіндексовано!"
    except Exception as e:
        return f"❌ Помилка індексації: {e}"

# ❓ Обробка запиту

def ask_question(question):
    if not qa_chain:
        return "❗ Завантажте документ спочатку!"
    try:
        # Витягуємо номер сторінки
        import re
        m = re.search(r"(?:page|стр)\s*(\d+)", question, re.IGNORECASE)
        if m:
            p = int(m.group(1))
            text = page_texts.get(p, "")
            if not text:
                return f"❌ Сторінку {p} не знайдено."
            # Використовуємо summarize chain
            docs = [Document(text, metadata={"page": p})]
            llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.3, max_tokens=512)
            sum_chain = load_summarize_chain(llm, chain_type="map_reduce")
            summary = sum_chain.run(docs)
            return f"**🧠 Summary of page {p}:**\n{summary.strip()}"
        # Інакше RAG
        res = qa_chain.invoke({"query": question})
        ans = res["result"]
        srcs = "\n\n---\n\n".join([f"📄 Page {d.metadata.get('page')}\n{d.page_content[:200]}..." for d in res["source_documents"]])
        return f"**🧠 Answer:**\n{ans}\n\n📚 **Sources:**\n{srcs}"
    except Exception as e:
        return f"❌ Помилка: {e}"

# 🎛️ Gradio UI

def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## 📘 RAG Assistant (GPT-4.1 via OpenRouter)")
        with gr.Row():
            fi = gr.File(label="Завантаж PDF або TXT")
            btn = gr.Button("📥 Індексувати")
        status = gr.Textbox(label="Статус")
        btn.click(upload_file, inputs=fi, outputs=status)
        with gr.Row():
            q = gr.Textbox(label="Питання або 'page X'")
            ask = gr.Button("🤖 Запитати")
        out = gr.Markdown()
        ask.click(ask_question, inputs=q, outputs=out)
    return demo

# 🚀 Запуск
demo = create_interface()
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b4bddb4f2b3e2130a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
!pip install PyPDF2 pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 354.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.1 MB/s eta 0:00:00
